In [26]:
# -*- coding: utf-8 -*-
import warnings
import pandas as pd
import numpy  as np
from sklearn.preprocessing import StandardScaler,Imputer
from sklearn.feature_selection import RFECV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib

#ワーニング非表示
warnings.filterwarnings('ignore') 

# csvファイルを読み込む
df_data = pd.read_csv('data.csv', header=0, quotechar='"', encoding='cp932')

# csvファイルの読み込みの正常終了を確認する
display(df_data.head(5), df_data.shape)


,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


(9532, 16)

In [27]:
# 特徴量Xと正解ラベルyの設定
X = df_data.iloc[:, 0:-1]
y = df_data.iloc[:, -1]

# 特徴量Xと正解ラベルyの設定の正常終了を確認する
display(X.head(5), X.shape)
display(y.head(5), y.shape)

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916


(9532, 15)

0    46135
1    40650
2    36350
3    29450
4    34500
Name: MSRP, dtype: int64

(9532,)

In [28]:
# 2017年を基準に車両が販売されてからの経過年数を新しい特徴量として生成する
X['Duration Since Production'] = 2017 - X['Year']
display(X.head(5))

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,Duration Since Production
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,6
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,6
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,6
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,6
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,6


In [29]:
#カテゴリカル変数を指定
ohe_cols = ['Make','Model','Engine Fuel Type','Transmission Type','Driven_Wheels','Market Category','Vehicle Size','Vehicle Style']

In [30]:
#プロセス1：one-hot encoding
#カテゴリ変数を0、1にし、欠損処理を行う
X_ohe = pd.get_dummies(X, dummy_na=True, columns=ohe_cols)
print('one-hot encoding 実施後 shape:(%i,%i)' % X_ohe.shape)
#全ての要素がnullの列を削除
X_ohe = X_ohe.dropna(axis=1, how='all')
print('全ての要素がnullの列を削除後 shape:(%i,%i)' % X_ohe.shape)

one-hot encoding 実施後 shape:(9532,1070)
全ての要素がnullの列を削除後 shape:(9532,1070)


In [31]:
#カラム取得
X_ohe_columns = X_ohe.columns.values

# プロセス2: null imputation　連続変数の欠損処理
#null値を平均値で補完
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
#作成したインピュータにfitさせる
imp.fit(X_ohe)
#transformでX_oheを書き換え
X_ohe = pd.DataFrame(imp.transform(X_ohe), columns=X_ohe_columns)
print('連続変数の欠損処理後 shape:(%i,%i)' % X_ohe.shape)

連続変数の欠損処理後 shape:(9532,1070)


In [44]:
# プロセス3: 標準化
#ランダムフォレストでは標準化しない

# プロセス4： feature selection　次元圧縮
#RFECVにて特徴量選択を行う
selector = RFECV(estimator=RandomForestClassifier(random_state=0),step=0.05)
print('終了')

終了


In [45]:
#作成したセレクタにfitさせる
selector.fit(X_ohe, y.as_matrix().ravel())
print('終了')

終了


In [46]:
#transformでX_oheを書き換え
X_ohe_selected = selector.transform(X_ohe)
print('終了')

終了


In [47]:
#セレクタにて抽出
X_ohe_selected = pd.DataFrame(X_ohe_selected, columns=X_ohe_columns[selector.support_])
print('セレクタ抽出後 shape:(%i,%i)' % X_ohe_selected.shape)
#セレクタ抽出後、データ表示（headのみ）
display(X_ohe_selected.head())

セレクタ抽出後 shape:(9532,540)


,Year,Engine HP,Engine Cylinders,Number of Doors,highway MPG,city mpg,Popularity,Duration Since Production,Make_Acura,Make_Aston Martin,...,Vehicle Style_Convertible,Vehicle Style_Convertible SUV,Vehicle Style_Coupe,Vehicle Style_Crew Cab Pickup,Vehicle Style_Extended Cab Pickup,Vehicle Style_Passenger Minivan,Vehicle Style_Passenger Van,Vehicle Style_Regular Cab Pickup,Vehicle Style_Sedan,Vehicle Style_Wagon
0,2011.0,335.0,6.0,2.0,26.0,19.0,3916.0,6.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2011.0,300.0,6.0,2.0,28.0,19.0,3916.0,6.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2011.0,300.0,6.0,2.0,28.0,20.0,3916.0,6.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2011.0,230.0,6.0,2.0,28.0,18.0,3916.0,6.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2011.0,230.0,6.0,2.0,28.0,18.0,3916.0,6.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
# スコアデータロード
dfs = pd.read_csv('score.csv', header=0, quotechar='"', encoding='cp932')
#ID列取得
IDs = dfs.iloc[:,[0]]
#X取得
Xs = dfs.iloc[:, :]
#プロセス1：one-hot encoding
#カテゴリ変数を0、1にし、欠損処理を行う
Xs_ohe = pd.get_dummies(Xs, dummy_na=True, columns=ohe_cols)
print('スコアデータのone-hot encoding 実施後 shape:(%i,%i)' % Xs_ohe.shape)

#テストデータのカラムのみデータフレームで取得
cols_m = pd.DataFrame(None, columns=X_ohe_columns, dtype=float)
print('テストデータのカラムのみデータフレーム shape:(%i,%i)' % cols_m.shape)

#一旦検証***
#モデルの列とスコアの列を取得
cols_model = set(X_ohe.columns.values)
cols_score = set(Xs_ohe.columns.values)

# モデルにはあったがスコアにはないデータ項目
diff1 = cols_model - cols_score
print('モデル サイズ:(%i)' % len(cols_model))
print('モデルのみに存在する項目 サイズ:(%i)' % len(diff1))
#print('モデルのみに存在する項目: %s' % diff1)

# スコアにはあるがモデルになかったデータ項目
diff2 = cols_score - cols_model
print('スコア サイズ:(%i)' % len(cols_score))
print('スコアのみに存在する項目 サイズ:(%i)' % len(diff2))
#print('スコアのみに存在する項目: %s' % diff2)

# consistent with columns set
#スコアのみに存在する項目を追加
#縦に結合
Xs_exp = pd.concat([cols_m, Xs_ohe])
print('スコアのみに存在する項目を追加後 shape:(%i,%i)' % Xs_exp.shape)
#スコアのみに存在する項目をゼロ埋め(naの場合、0に変換)
Xs_exp.loc[:,list(set(X_ohe_columns)-set(Xs_ohe.columns.values))] = \
  Xs_exp.loc[:,list(set(X_ohe_columns)-set(Xs_ohe.columns.values))].fillna(0, axis=1)
print('スコアのみに存在する項目をゼロ埋め(naの場合、0に変換)後 shape:(%i,%i)' % Xs_exp.shape)
#モデルのみに存在する項目を削除
Xs_exp = Xs_exp.drop(list(set(X_ohe_columns)-set(Xs_ohe.columns.values)), axis=1)
print('モデルのみに存在する項目を削除後 shape:(%i,%i)' % Xs_exp.shape)

# re-order the score data columns
#モデリング時点のデータ項目の並び順を明示的に担保する
#reindex 列の並び順を変える（モデリング時点のone-hotエンコーディング後の並び順にする）
Xs_exp = Xs_exp.reindex_axis(X_ohe_columns, axis=1)

#並び替えが完了したら、モデリング時と同様に処理を行う。
#モデリング時に作成したインピューターにてスコアリングデータの欠損処理を行う。
Xs_exp = pd.DataFrame(imp.transform(Xs_exp), columns=X_ohe_columns)
#標準化は行わない（モデリング時に行わなかったため。）
#モデリング時に作成したセレクタで使用する列を選択する。（次元圧縮を行う。）
Xs_exp_selected = Xs_exp.loc[:,X_ohe_columns[selector.support_]]
#　！！！スコアデータ作成完了！！！

スコアデータのone-hot encoding 実施後 shape:(2382,860)
テストデータのカラムのみデータフレーム shape:(0,1070)
モデル サイズ:(1070)
モデルのみに存在する項目 サイズ:(228)
スコア サイズ:(860)
スコアのみに存在する項目 サイズ:(18)
スコアのみに存在する項目を追加後 shape:(2382,1088)
スコアのみに存在する項目をゼロ埋め(naの場合、0に変換)後 shape:(2382,1088)
モデルのみに存在する項目を削除後 shape:(2382,860)


In [61]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
# pipeline setting
pipelines = {
     'ols': Pipeline([('scl',StandardScaler()),
                      ('est',LinearRegression())]),
     
     'ridge':Pipeline([('scl',StandardScaler()),
                       ('est',Ridge(random_state=0))]),

     'rf': Pipeline([('scl',StandardScaler()),
                     ('est',RandomForestRegressor(random_state=0))]),
     
     'gbr1': Pipeline([('scl',StandardScaler()),
                      ('est',GradientBoostingRegressor(random_state=0))]),

     'gbr2': Pipeline([('scl',StandardScaler()),
                      ('est',GradientBoostingRegressor(n_estimators=250,
                                                       random_state=0))])
}

# build and evaluate
scores = {}
for pipe_name, pipeline in pipelines.items():
    pipeline.fit(X_ohe_selected, y.as_matrix().ravel())
    scores[(pipe_name,'結果')] = r2_score(y.as_matrix().ravel(), pipeline.predict(X_ohe_selected))

pd.Series(scores).unstack()

,train
gbr1,0.961795
gbr2,0.977392
ols,0.765483
rf,0.980635
ridge,0.764235
